In [15]:
from dotenv import load_dotenv
import os
from ohlc_data_process import calculate_dmi_rsi_mfi, fetch_data

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

In [19]:
from langchain import OpenAI

openai = OpenAI(
    model_name='text-davinci-003',
    #model_name='text-curie-001',
    temperature=0.7
)

In [20]:
from langchain import PromptTemplate

template = """Answer the question based on the context below.
You are the trading advisor. Also you are expert on RSI, MFI and DMP, DMI indicators.

Context: RSI indicator value range betwwen 0-100. RSI value 70 and above meaning that overbought area.
 So you should sell your coin. also value 80 is a extreme overbought and you act cautiously.
 RSI value 30 that meaning is overselling area. and value 20 is extreme overselling are.
 if the RSI value are 20-30s you should bought this coin. 30-70 range waiting or
 you can look other indicator results.
 MFI indicator value range betwen 0-100. MFI value 80 and above meaning that overbought area.
 So you should sell your coin. also value 90 is a extreme overbought and you should act cautiously.
 MFI value 20 that meaning is overselling area. and value 10 is extreme overselling are.
 if the MFI value are 10-20s you should bought this coin. 20-80 range waiting or
 you can look other indicator results.
 Else it shows overselling condition between 0-25.
 DMI indicator is a collection of indicators including DMP, DMI, and ADX. The Plus Direction Indicator DMP and
 Minus Direction Indicator DMI show the current price direction. When the DMP is above DMN,
 the current price momentum is up. When the DMN is above DMP, the current price momentum is down.
 ADX measures the strength of the trend, either up or down; a reading above 25 indicates a strong trend.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [21]:
df = fetch_data('BTC-USD', '3mo', '1d')
indicator_data = calculate_dmi_rsi_mfi(df)
rsi_14 = indicator_data.RSI_14.iloc[-1]
mfi_14 = indicator_data.MFI_14.iloc[-1]
dmp_14 = indicator_data.DMP_14.iloc[-1]
dmn_14 = indicator_data.DMN_14.iloc[-1]
adx_14 = indicator_data.ADX_14.iloc[-1]

print(openai(
    prompt_template.format(
        query=f"RSI indicator value is {rsi_14:.2f}, MFI indicator value is {mfi_14:.2f}, DMP indicator value is {dmp_14:.2f},"
              f" DMN indicator value is {dmn_14:.2f} and ADX indicator value is {adx_14:.2f}."
              f" What is your advice for trading for those indicator values?"
    )
))

 Based on the given indicator values, it appears that the RSI and MFI indicators are in a neutral range (30-70 for RSI and 20-80 for MFI). The DMP and DMN values indicate that the current price momentum is down, and the ADX value indicates that there is a weak trend. Thus, it is best to wait for a clearer indication from the indicators before making any trading decisions.


In [11]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "RSI_10 daily value is 35 and EMA 5 daily value above than EMA 10 daily. What is your Advice for trading?",
        "answer": "The RSI value is near the overselling if youe are hodl any coin i think you continue to hold this coin." 
                "otherwise you can wait for the RSI decreasing for good buying oppurtunity and EMA value says bullish trend is continueing. You can think about bullish position."
    }, {
        "query": "RSI 10 daily value is 85 and EMA 7 daily value under than EMA 10 daily."
                "What is your Advice for trading?",
        "answer": "RSI value say this condition is overbought and you can sell any asset if you are hold any coin." 
                "Also EMA indicator shows a coin is in Bearish condition and you could sell if you are hold this coin."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are experts from conversations with an AI
trading advisor assistant.
The RSI and EMA indicator values are playing important role when trading on a daily basis.
 RSI data indicating a decline can indicate an overbought condition between 70-100. so you can sell your coin.
 Else it shows overselling condition between 0-25. so you can buy this coin.
 Overselling condition create a good oppurtunity for buyimg asset and overbought condition means that you should sell your asset.
 The EMA indicators generally says if lower daily period is
 above on higher daily period that means is bullish. if the condition is opposite for EMA than
 we can say that is bearish signal for selling assets.
Here are some examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [13]:
query = "RSI_10 daily value is 25 and EMA 6 daily value above than EMA 10 daily. What is your Advice for trading?"

print(openai(
    few_shot_prompt_template.format(query=query)
))



Since the RSI value is below the oversold territory, it may be a good time to sell your assets. Meanwhile, the EMA indicator suggests that the market is bullish, and so you may want to buy assets at this time.
